In [5]:
import csv, sys
import re
from collections import Counter
from datetime import date, datetime

# CSV now generated from Google sheet ("Research IT Consulting Engagement Log"), 
# then filtered for date, RIT Service (= 'RDM') and Category 
# (= consultation-related categories: 'user support' + all categories beginning 'RDM')

Provide some parameters (arguments) for the script:
1. The .csv file that contains our data is now being generated from the Google sheet titled "Research IT Consulting Engagements Log." 
2. The 'report_period_descriptor' will be the text included in the first line of the report created by the script.
3. Report_start and report_end are the beginning and end dates (inclusive) of the period covered by the report.

You can load these from a file if you wish. The file should be a .py file that contains exactly the information shown below in lines 2 through 7. Place the file in the same folder as this Jupyter Notebook file, or provide the path to it in the cell magic command '%load.' (Remove the '#' before '%load' to run the command.)

In [11]:
# %load Metrics_args/RDM_metrics_args-FY2018-Q1.py
# These are the arguments provided to the script
filepath = '/Users/rjaffe/Documents/RDM/RDM_Metrics/MetricsData/RITConsultingEngagementLog_Google-downloads/'
filename = 'Research-IT-Consulting-Engagements-Log_20171019_1544PDT.csv'
report_period_descriptor = 'the first quarter of FY 2017-2018 (July 1 through September 30, 2017)'
report_start = '2017-07-01'
report_end = '2017-09-30'

To help keep us humans from confusion, here's a list of the column headers in the original Google sheet. We will refer to these "columns" in the script by their position in each row, with the first column being element [0].

In [12]:
# Google sheet column headers (row to be removed by the Python script):
#    [0] Start date
#    [1] Consultant(s)
#    [2] Client(s)
#    [3] PI (Whose project is it?)
#    [4] Department/ORU
#    [5] Research Domain (e.g. Egyptology)
#    [6] Position (grad, postdoc, faculty, undergrad, researcher)
#    [7] Project type (dissertation, etc.)
#    [8] Related course (if applicable)
#    [9] Research IT service
#    [10] Topic (uncontrolled)
#    [11] Category (controlled)
#    [12] Source
#    [13] Hand-off and/or referral
#    [14] Complexity (RDM)
#    [15] Status
#    [16] Link to details
#    [17] Notes
#    [18] (empty)

This script utilizes dictionaries to roll up individual values to a parent value. At this point, there are five:
1. The "columns" of the .csv file that will be counted or analyzed using dictionaries
2. The organizational unit of people involved in consultations or referrals
3. The departments served belong to colleges, divisions; etc. (To be added from the Securing Research Data work)
4. The labels used in our output for empty values; the labels vary by column
5. The headings used in our output; these, like the labels, vary by column.

Before we load the data file, let's initialize some configuration for the various dictionaries. The configuation is kept in a file currently named config_v3.py, which we load using the cell magic command '%load.' (The file can be located in the same folder as the notebook file itself, or a path can be provided.) 

As new values are added to the Google Sheet, they should be added to the second value list within the correct dictionary here.

In [13]:
# %load config_v3.py  # file location is relative to this Jupyter Notebook
# Positional elements (columns) with the following indices -- and only those elements, at present --
# will be gathered and counted using a dictionary
dictable_cols = [1, 4, 6, 11, 12, 13, 14]

# let the data do the heavy lifting...everything is in this dict!
refs = {
    'lib': ['The Library', ['Harrison Dekker', 'Jamie Wittenberg', 'Susan Edwards', 'Steve Mendoza', 'Steven Mendoza',
                            'Margaret Phillips', 'data-consult list (Library)', 'Brian Quigley', 'Library',
                            'Data Storage/Sharing and the Social Sciences Working Group', 'Erik Mitchell',
                            'Susan Powell', 'Anna Sackmann', 'David Eiffler', 'Yasmin Alnoamany', 'Stacy Reardon',
                            'Celia Emmelhainz', 'Hilary Schiraldi', 'Amy Neeser']],
    'css': ['Campus Shared Services - IT', ['Brett Larsen', 'Daniel Bass', 'Johnathon Kogelman',
                                            'Johnathon Kogelman (CSS-IT)', 'CSS-IT', 
                                            'Referred by Johnathon Kogelman (CSS-IT)', 
                                            'request to Rick from Daniel Bass',
                                            'email to Rick from Daniel Bass',
                                            'CSS_IT (Jon Valmores)']],
    'dlab': ['D-Lab', ['D-Lab Consulting List', 'D-Lab', 'Jon Stiles', 'Zawadi Rucks Ahidiana',
                       'Rick Jaffe (via D-Lab Consulting web page)', 'dlab-consultants@lists.b.e', 
                       'd-lab consultants list', 'referred to D-Lab/Jon Stiles', 'D-Lab consultants list',
                       'Rick at d-lab consulting Ticket #29430', 'D-Lab ticket#29433', 'Chris Hench (D-Lab)']],
    'scf': ['Statistical Computing Facility', ['Chris Paciorek', 'Ryan Lovett']],
    'brc': ['Berkeley Research Computing',
            ['Patrick Schmitz', 'Aron Roberts', 'Aaron Culich', 'Jason Christopher', 'Kelly Rowland', 'Gary Jung',
             'BRC Cloud Consulting', 'Jason Huff (Computational Genomics Resource Lab)',
             'Berkeley Research Computing - Cloud', 'Yong Qin', 'Deb McCaffrey', 'email to BRC', 
             'brc@berkeley.edu']],
    'dh': ['Digital Humanities @ Berkeley', ['Quinn Dombrowski', 'Camille Villa', 'Digital Humanities',
                                             'Claudia Natalia Von Vacano']],
    'rdm': ['RDM Consulting', ['researchdata@berkeley.edu', 'Rick Jaffe', 'Chris Hoffman', 'John B Lowe',
                               'BRC Survey 2016 (Response to follow-up from Jamie)', 'email to Rick Jaffe', 
                               'Follow-up', 'Rick', 'follow up', 'email to Rick and Jason', 
                               'researchdata@b.e. (after browsing web site)', 'researchdata@b.e.', 
                               'researchdata@b.e', 'email to Rick from Jessica', 'email to Rick from Carla',
                               'email to Rick from Laura', 'email to Rick from Phuong', 'email to Rick from Sarah',
                               'Anna Sackman (RDM)', 'Email to Rick']],
    'cdl': ['California Digital Library', ['Joan Starr', 'Stephanie Simms', 
                                           'Daniella Lowenberg (DASH), via Quinn Dombrowski']],
    'ist': ['Information Services & Technology - API', ['Jennifer Bellenger', 'Jon Broshious', 'Ian Crew', 'Jon Hays',
                                                        'bConnected', 'Michael Leefers', 'Alex Walton', 
                                                        'referred by Ian (bConnected)', 
                                                        'referred to Rick by Jennifer Bellenger (bConnected)',
                                                        'Forwarded by Beth Muramoto (GSE) to Ian Crew (bConnected), who forwarded it in turn to researchdata@berkeley.edu',
                                                        'bconnected']],
    'micronet': ['Micronet', ['micronet', 'Micronet', 'micronet list', 'Micronet list']],
    'iao': ['Industry Alliances Office', ['Nicole Hensley', 'Nicole Hensley (IAO/IPIRA)', 'Nicole Hensley (IAO)',
                                         'email to Chris and Rick from Eric Giegerich',
                                         'email from Nicole Hensley to Rick and Chris',
                                         'Email from Nicole Hensley to Chris and to Rick']],
    'ssw': ['School of Social Welfare', ['David Fullmer']],
    'bids': ['Berkeley Institute for Data Science', ['BIDS']],
    'brdo': ['Berkeley Research Development Office (VCRO)', ['Barbara Ustanko via Chris Hoffman']],
    'lsit': ['Letters & Science IT', ['Michael Quan (Letters & Science IT)']],
    'ais': ['Academic Innovation Studio', ['AIS drop-in (handled by Rick)']],
    'musinf': ['Museum Informatics', ['BIDS Faire CSpace Portals poster']],
    'rit': ['Research IT', ['research-it@berkeley.edu']]
}

labels = ['Unassigned', 'Unknown department', 'Unknown status', '',  '', '', 'Unspecified', 'Unknown division',
          'Consultation(s) without a partner',]

headings = ['Consultants, number of consults', 'Departments Served, number of engagements',
            'Patron Status, number of patrons', 'RDM Lifecycle Category', 'Referrals In', 'Referrals Out',
            'Consultation Complexity', 'Library Division, number of engagements',
            'Organizational Partners, number of shared engagements' ]


CLEAN THE DATA

With the dictionaries in place, let's begin to clean the data.

In [18]:
# TODO - Catch and handle missing arguments or errors in the arguments

# Convert report_start and report_end arguments to datetime format
reportstart = datetime.strptime(report_start, '%Y-%m-%d')
reportend = datetime.strptime(report_end, '%Y-%m-%d')

myrows = []

# Read data into a list of lists, clean as required
with open(filepath + filename) as csvfile:
    for row in csv.reader(csvfile, delimiter=","):

        # Filter Google sheet to include only RDM consultations during the desired period.
        
        # Remove header row (first header value is 'Start Date')
        if row[0] == 'Start Date': continue

        
        # Convert start date values (first column) to datetime format and \
        # compare against report-start and report-end arguments. Skip if start date is not in report period range
        startdate = datetime.strptime(row[0], '%Y-%m-%d')
        if not reportstart <= startdate <= reportend: continue
            
        # Remove rows in which Research IT Service does not include RDM
        RIT_service = row[9]
        if not 'RDM' in RIT_service: continue

        # Remove rows that are not consultations. For RDM, consultations were listed as 'User support' \
        # or (once) 'Library user support' until late February 2017. \
        # After that, they were coded as 'RDM [service area]', sometimes with multiple values listed
        p = re.compile(r'^.*[Uu]ser support.*$')  # Matches 'User support' or 'Library user support'
        p1 = re.compile(r'^.*(RDM)')  # Matches an instance of 'RDM [service area]'
        category = row[11]
        if not ((p.match(category)) or (p1.match(category))): continue
 

        # Now clean, split multiple values, and aggregate (roll up) values as appropriate
        
        # Consultant(s), Department/ORU, Patron status, (RDM Lifecycle) Category, Source (aka referral in),
        # Hand-off or referral (aka referral out), Consultation complexity: \
        # replace empty values with appropriate label
        # NOTE: We didn't port Library division and Organizational partner fields to the Google sheet
        
        #for n, label in zip(config_v2.dictable_cols, config_v3.labels):  ## USE THIS IN PYCHARM
        for n, label in zip(dictable_cols, labels):

            # Fill in empty cells with appropriate label
            if row[n] == '':
                row[n] = label
                    
            # Remove trailing soft returns (i.e.,\n) -- it's hard to control these in Google Sheets.
            val = row[n]
            suffix = '\n'
            if(val.endswith(suffix)):
                val = val[:-1]
                row[n] = val  # I don't completely trust this, but I don't seem to be losing any data!
            
            # make every cell into a list (some cells have new-line separated values)
            row[n] = row[n].split('\n')
        # Replace individual names with the corresponding org name in Source (aka Referral In) and
        # Hand-off or referral (aka Referral Out) fields
        # (positional elements [12] and [13])
        for n in [12, 13]:
            ref_x = row[n]
            row[n] = []  # Empty cell to ready it for being re-filled
            for term in ref_x:
                #for key in config_v3.refs.keys():    ## USE THIS IN PYCHARM
                for key in refs.keys():
                    #if term in config_v3.refs[key][1]:   ## USE THIS IN PYCHARM
                    if term in refs[key][1]: 
                        #term = config_v3.refs[key][0]    ## USE THIS IN PYCHARM
                        term = refs[key][0]
                        row[n].append(term)
        myrows.append(row)

In [19]:
# ***** COUNT THE DATA *****
# Each row (list) represents a consulting engagement
print('\nIn %s, RDM Consulting provided %d consultations.' % (report_period_descriptor, (len(myrows))))

# Count how many engagements are resolved successfully
yesrows = []

for r in myrows:
    r15 = r[15]
    if 'Resolved' in r15:
        yesrows.append(r)

print('We reached a successful resolution in %d of those engagements.' % len(yesrows))



In the first quarter of FY 2017-2018 (July 1 through September 30, 2017), RDM Consulting provided 28 consultations.
We reached a successful resolution in 2 of those engagements.


In [20]:
# Gather and count (subtotal) the values for consultant(s), department/oru, patron status, (RDM lifecycle) category,
# source (referrals in), hand-off or referral (referrals out) and consultation complexity.
# TODO: calculate values for library division and organizational partners fields
#for i, n in enumerate(config_v3.dictable_cols):   ## USE THIS IN PYCHARM
for i, n in enumerate(dictable_cols):
    counter = Counter()
    for row in myrows:
        for z in row[n]:
            counter[z] += 1
    #print('\n' + config_v3.headings[i] + ':')   ## USE THIS IN PYCHARM
    print('\n' + headings[i] + ':')
    for (k, v) in counter.most_common():
        print(k + ', ' + str(v))


Consultants, number of consults:
Rick Jaffe, 19
Chris Hoffman, 8
Aaron Culich, 3
Josh Quan, 2
Maurice Manning, 2
Chris Doane, 1
Oliver Muellerklein, 1
Unassigned, 1
Patrick Schmitz, 1
David Rieger (IST), 1
Blaine Isbelle (IST), 1
Joe Silva (IST), 1
Jonathon Taylor (IST), 1
Amy Neeser, 1
Kortney Rupp, 1
Anna Sackmann, 1

Departments Served, number of engagements:
Unknown department, 4
Department of Integrative Biology, 2
Institute of European Studies, 2
Department of Music, 2
School of Law, 2
Berkeley Institute for Data Science (BIDS), 2
Haas Institute for a Fair & Inclusive Society, 2
Department of Psychology, 2
Division of Biostatistics/Public Health, 2
Department of Sociology, 1
College of Natural Resources (CNR), 1
Digital Humanities at Berkeley (DH), 1
School of Public Health, 1
Department of Plant and Microbial Biology, 1
Industry Alliances Office (Vice Chancellor for Research), 1
Research IT, 1
Department of Bioengineering, 1
Department of Anthropology, 1
Department of Agricultu